## Inspired by https://www.kaggle.com/awsaf49/great-barrier-reef-yolov5-train 

In [1]:
#Import Dependencies
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import shutil
import warnings
import yaml
warnings.filterwarnings("ignore")
os.environ['KMP_DUPLICATE_LIB_OK']='True'

%matplotlib inline

start=time.time()

In [2]:
#path of folder with downloaded data
base_path_data="C:/Users/soyrl/Desktop/tensorflow-great-barrier-reef/" 

In [3]:
#Splits for train and validation
train=['video_0','video_1']
val=['video_2']

In [4]:
train_csv_path=base_path_data+"train.csv" #Set path to train.csv file
train_csv=pd.read_csv(train_csv_path) #Load train_csv file to dataframe

In [5]:
print(len(np.unique(train_csv['sequence'].values))) #Show number of unique sequences

20


In [6]:
train_csv[:-1] #Show last rows of dataframe to compare video_frames and sequence

,video_id,sequence,video_frame,sequence_frame,image_id,annotations
0,0,40258,0,0,0-0,[]
1,0,40258,1,1,0-1,[]
2,0,40258,2,2,0-2,[]
3,0,40258,3,3,0-3,[]
4,0,40258,4,4,0-4,[]
...,...,...,...,...,...,...
23495,2,29859,10754,2982,2-10754,[]
23496,2,29859,10755,2983,2-10755,[]
23497,2,29859,10756,2984,2-10756,[]
23498,2,29859,10757,2985,2-10757,[]


In [7]:
#Create new column with path of training images 
train_csv['path']=base_path_data+"train_images/"+"video_"+train_csv['video_id'].astype(str)+'/'+train_csv['video_frame'].astype(str)+'.jpg'

In [8]:
#Create new column with path of training images 
# train_csv['path_id']=base_path_data+"train_images/"+"video_"+train_csv['video_id'].astype(str)+'/'+train_csv['video_frame'].astype(str)+'.jpg'

In [9]:
train_csv['path'][1000] #Check random path

'C:/Users/soyrl/Desktop/tensorflow-great-barrier-reef/train_images/video_0/1393.jpg'

In [10]:
#Create path to save annotations in YOLOv5 format
if not os.path.exists(os.getcwd()+"/texts"):
    os.makedirs("texts")

In [11]:
#Make subdirs to save annotations in YOLOv5 format
if not os.path.exists(os. getcwd()+"/texts/video_0"):
    os.makedirs(os. getcwd()+"/texts/video_0")
if not os.path.exists(os. getcwd()+"/texts/video_1"):
    os.makedirs(os. getcwd()+"/texts/video_1")
if not os.path.exists(os. getcwd()+"/texts/video_2"):
    os.makedirs(os. getcwd()+"/texts/video_2")

### Reef file prepared based on https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data

In [12]:
#Create new columns with number of boxes and box annotations
train_csv['num_boxes']='' 
train_csv['annot_yolov5']=''

train_csv['num_boxes']=train_csv["annotations"].apply(lambda annotations: len(eval(annotations))) #Add num of boxes in that col

for i in range(len(train_csv)): #Loop over rows of dataframe
    annot=eval(train_csv['annotations'].iloc[i]) #To convert string to list
    annot_str='' #Initialize empty string to be filled with annotations in the form 'x1 y1 w1 h1 x2 y2 w2 h2 ....' in YOLO format
    annot_str2='' #Same but now YOLO format (not normalization etc.) - Just to show an image with bounding boxes below
    lines=[] #List to be filled with string of bounding boxes in YOLO format for each image
    
    for j in range(train_csv['num_boxes'].iloc[i]): #Loop over the number of unique boxes in an image
        if annot!=[]: #If we have annotations
            x_center=float(annot[j]['x'])+float(annot[j]['width']/2) #Calculate center of bounding box
            y_center=float(annot[j]['y'])+float(annot[j]['height']/2)
            annot_str='0 '+str(x_center/1280)+' '+str(y_center/720)+' '+str(annot[j]['width']/1280)+' '+str(annot[j]['height']/720)+' '
            #Annotation in the YOLOv5 format
            
            annot_str2=str(annot_str2)+str(annot[j]['x'])+' '+str(annot[j]['y'])+' '+str(annot[j]['width'])+' '+str(annot[j]['height'])+' '
            #Annotations just for demonstration purposes
            
            if x_center/1280>1 or y_center/720>1 or annot[j]['width']/1280>1 or annot[j]['height']/720>1:
                print("problem with image {}".format(train_csv['annotations'].iloc[i]))
                print("index is {}".format(i))
                
        else: #if annotations do not exist
            annot_str=''
                        
        lines.append(annot_str) #Append all the annotations for a given image
                
    train_csv['annot_yolov5'].iloc[i]=annot_str2 #Add final annotation to the corresponding column - Just for demonstration
##!!!!!!
    if len(lines)!=0 and i!=5293: #Ignore file with bounding box boundaries out of image
        #Create empty txt files to be filled with annotations 
        file1=open(os.path.join(os. getcwd()+"/texts/", train_csv['path'][i][:-4].split('/')[-2]+'/'+train_csv['path'][i][:-4].split('/')[-1]+'.txt'),'w')
#         file1=open(os.path.join(os. getcwd()+"/texts/", train_csv['path'][i][:-4].split('/')[-2]+'/'+train_csv['image_id'][i]+'.txt'),'w')
        file1.close()
        
        #Add annotations to the txt files in the YOLOv5 format
#         with open(os. getcwd()+"/texts/"+train_csv['path'][i][:-4].split('/')[-2]+'/'+train_csv['image_id'][i]+'.txt', 'w') as f:
#                for line in lines:
#                     f.write(line)
#                     f.write('\n')
        with open(os. getcwd()+"/texts/"+train_csv['path'][i][:-4].split('/')[-2]+'/'+train_csv['path'][i][:-4].split('/')[-1]+'.txt', 'w') as f:
            for line in lines:
                f.write(line)
                f.write('\n')

problem with image [{'x': 286, 'y': 704, 'width': 85, 'height': 35}, {'x': 631, 'y': 652, 'width': 55, 'height': 42}]
index is 5293


In [17]:
#Make dirs to save train and val images
if not os.path.exists(os. getcwd()+"/texts/train"):
    os.makedirs(os. getcwd()+"/texts/train")
if not os.path.exists(os. getcwd()+"/texts/val"):
    os.makedirs(os. getcwd()+"/texts/val")

In [19]:
#Create train and val dirs
for dir in train:
    if not os.path.exists(os. getcwd()+"/texts/train/"+dir):
        os.makedirs(os. getcwd()+"/texts/train/"+dir)
for dir in val:
    if not os.path.exists(os. getcwd()+"/texts/val/"+dir):
        os.makedirs(os. getcwd()+"/texts/val/"+dir)

In [20]:
#Copy train and val annotations to the folders required by YOLOv5 and delete source folders
for dir in os.listdir(os.path.join(os. getcwd()+"/texts/")):
    if dir in train:
        #Copy annotations to the folder required by YOLO
        source=os.getcwd()+"/texts/"+dir+'/'
        destination=os.getcwd()+"/texts/"+"train/"+dir
        from distutils.dir_util import copy_tree
        copy_tree(source, destination)
        shutil.rmtree(source)
    if dir in val:
        #Copy annotations to the folder required by YOLO
        source=os.getcwd()+"/texts/"+dir+'/'
        destination=os.getcwd()+"/texts/"+"val/"+dir
        from distutils.dir_util import copy_tree
        copy_tree(source, destination)
        shutil.rmtree(source)

In [25]:
#Change path in reef.yaml file according to our structure
with open(os.getcwd()+'/'+"reef.yaml", "r", encoding="utf8") as file:#stream:
    try:
        loaded=yaml.load(file)
#         print(loaded)
    except yaml.YAMLError as exc:
        print(exc)

train_fin='\n'
val_fin='\n'

for elem in train:
    train_fin=train_fin+'- images/train/'+elem+'\n' #-images
for elem2 in val:
    val_fin=val_fin+'- images/val/'+elem2+'\n' #-images
    
print(train_fin) 
print(val_fin)
print(loaded['train'])
print(loaded['val'])

loaded['path']=os.getcwd()+'/'+'datasets/reef'
loaded['train']=train_fin
loaded['val']=val_fin

with open(os.getcwd()+'/'+"reef.yaml", 'w') as stream:
    try:
        yaml.dump(loaded, stream, default_flow_style=False)
    except yaml.YAMLError as exc:
        print(exc)
        
index=0        
#Create a second txt file to temporary save information and then replace our original file
with open(os.getcwd()+'/'+"reef.yaml", 'r') as f, open('reef2.yaml', 'w') as fo:
    for line in f:
        if index==1:
            index=0
            continue
        else:
            fo.write(line.replace('"', '').replace("'", ""))
        
with open(os.getcwd()+'/'+"reef2.yaml", 'r') as f, open('reef.yaml', 'w') as fo:
    for line in f:
            fo.write(line)


- images/train/video_0
- images/train/video_1


- images/val/video_2

['images/train/video_0', 'images/train/video_1']
['images/val/video_2']


#### Clone the yolov5 repository, install requirements and confirm that GPU is found

In [29]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


In [30]:
%cd yolov5
%pip install -qr requirements.txt  # install

from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5  v6.0-206-gc43439a torch 1.10.1+cu113 CUDA:0 (GeForce GTX 1660 Ti, 6144MiB)


Setup complete  (12 CPUs, 31.8 GB RAM, 192.2/237.7 GB disk)


In [31]:
#Copy reef.yaml file inside yolo folder from github
source = "../reef.yaml" #The use of "../" in file path means 'go to previous directory'
destination = os. getcwd()+"/data/"
shutil.copy(source, destination)

'C:\\Users\\soyrl\\Desktop\\yolov5/data/reef.yaml'

In [32]:
#Create directories needed from YOLO
if not os.path.exists("../datasets"):
    os.makedirs("../datasets")
if not os.path.exists("../datasets/reef"):
    os.makedirs("../datasets/reef")
if not os.path.exists("../datasets/reef/images"):
    os.makedirs("../datasets/reef/images")
if not os.path.exists("../datasets/reef/labels"):
    os.makedirs("../datasets/reef/labels")

In [34]:
#Create train and val folders inside folder used by YOLOv5
if not os.path.exists("../datasets/reef/labels/train"):
    os.makedirs("../datasets/reef/labels/train")
if not os.path.exists("../datasets/reef/labels/val"):
    os.makedirs("../datasets/reef/labels/val")

In [35]:
#Create video subdir folders inside folder used by YOLOv5
for dir in train:
    if not os.path.exists("../datasets/reef/labels/train/"+dir):
        os.makedirs("../datasets/reef/labels/train/"+dir)
for dir in val:
    if not os.path.exists("../datasets/reef/labels/val/"+dir):
        os.makedirs("../datasets/reef/labels/val/"+dir)

In [36]:
cd ..

C:\Users\soyrl\Desktop


In [37]:
#Copy annotations to the folder required by YOLO
source=os. getcwd()+"/texts/"
destination=os. getcwd()+"/datasets/reef/labels/"
from distutils.dir_util import copy_tree
copy_tree(source, destination)

['C:\\Users\\soyrl\\Desktop/datasets/reef/labels/train\\video_0\\100.txt',
 'C:\\Users\\soyrl\\Desktop/datasets/reef/labels/train\\video_0\\1000.txt',
 'C:\\Users\\soyrl\\Desktop/datasets/reef/labels/train\\video_0\\1001.txt',
 'C:\\Users\\soyrl\\Desktop/datasets/reef/labels/train\\video_0\\1002.txt',
 'C:\\Users\\soyrl\\Desktop/datasets/reef/labels/train\\video_0\\1003.txt',
 'C:\\Users\\soyrl\\Desktop/datasets/reef/labels/train\\video_0\\1004.txt',
 'C:\\Users\\soyrl\\Desktop/datasets/reef/labels/train\\video_0\\1005.txt',
 'C:\\Users\\soyrl\\Desktop/datasets/reef/labels/train\\video_0\\1006.txt',
 'C:\\Users\\soyrl\\Desktop/datasets/reef/labels/train\\video_0\\1007.txt',
 'C:\\Users\\soyrl\\Desktop/datasets/reef/labels/train\\video_0\\1008.txt',
 'C:\\Users\\soyrl\\Desktop/datasets/reef/labels/train\\video_0\\1009.txt',
 'C:\\Users\\soyrl\\Desktop/datasets/reef/labels/train\\video_0\\101.txt',
 'C:\\Users\\soyrl\\Desktop/datasets/reef/labels/train\\video_0\\1010.txt',
 'C:\\Users\\s

### Problem below if we don't split based on 'video_i' folder, since images with the same names exist inside these folders

In [41]:
#Copy training images to the folder required by YOLO
image_path=base_path_data+"train_images/" #path to training images
destination=os.getcwd()+"/datasets/reef/images/"
from distutils.dir_util import copy_tree

if not os.path.exists(destination+'train'):
    os.makedirs(destination+'train')
if not os.path.exists(destination+'val'):
    os.makedirs(destination+'val')

for train_dir in train:
        copy_tree(image_path+train_dir+'/', destination+'train'+'/'+train_dir)
for val_dir in val:
    copy_tree(image_path+val_dir+'/', destination+'val'+'/'+val_dir)

In [42]:
%cd yolov5

C:\Users\soyrl\Desktop\yolov5


In [ ]:
#Set in the reef.yaml file the path to: current_working_dir+\datasets\reef

In [ ]:
# !python train.py --img 1280 --batch 4 --epochs 20 --data reef.yaml --weights yolov5s.pt --workers 2
#--device (0,1,.. find gpu!!), --resume --optimizer Adam --patience 10 --freeze --save-period 1

In [ ]:
Until here and then train. Hyperparameters from train modified based on
https://github.com/ultralytics/yolov5/issues/607
    
Training was performed on Peregrine. To run this jupyter notebook there use the following info
https://wiki.hpc.rug.nl/peregrine/examples/jupyter

#### The commands below are just to plot the image with the most annotations

In [ ]:
np.max(train_csv['num_boxes']) #Check the maximum number of boxes contained in an image

In [ ]:
np.where(train_csv['num_boxes']==np.max(train_csv['num_boxes'])) #Check indices of images with maximum number of boxes

In [ ]:
train_csv['path'][12679] #Get name of image with most boxes to be visualized below

In [ ]:
img=cv2.imread(train_csv['path'][12679]) #Load image with most annotations #12679
img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #Convert image back to RGB from BGR

box_num=train_csv['num_boxes'][12679] #get num of boxes for that image

for box in range(box_num): #Loop over num of boxes
    annotations=train_csv['annot_new'].iloc[12679].split() #Get each number as an element of a list
    cv2.rectangle(img,(int(annotations[4*box]),int(annotations[4*box+1])),
                  (int(annotations[4*box])+int(annotations[4*box+2]),int(annotations[4*box+1])+int(annotations[4*box+3])), 
                       (255,0,0), 2) #Add to the image one box at a time
    cv2.putText(img,'Starfish',(int(annotations[4*box]),int(annotations[4*box+1])),cv2.FONT_HERSHEY_DUPLEX,0.6,(0,0,255),1)
    #Put text above each rectangle
    
#Show image
plt.figure(figsize= (10,10)) 
plt.imshow(img)
plt.show()

In [ ]:
end=time.time()
print("Time to run is {} secs".format(end-start))

In [ ]:
train_csv.to_csv('train_csv_new.csv') #Save modified file

In [ ]:
example_test=np.load(base_path_data+'example_test.npy')
example_test.shape

In [ ]:
example_sub=pd.read_csv(base_path_data+'example_sample_submission.csv')
example_sub

## Decision Trees - Just for fun - Not Good

In [ ]:
# train_csv

In [ ]:
# train_csv['path'][train_csv['num_boxes']!=0]

In [ ]:
# data=np.zeros((int(len(train_csv['path'][train_csv['num_boxes']!=0])/50),720,1280),dtype=np.uint8) #was /5

In [ ]:
# data.shape

In [ ]:
# train_csv['path'][train_csv['num_boxes']!=0][:data.shape[0]]

In [ ]:
# for index,path in enumerate(train_csv['path'][train_csv['num_boxes']!=0][:data.shape[0]]):
# #     print(path)
#     img=cv2.imread(path) #Load image
# #     print(img.shape)
# #     plt.imshow(img)
# #     plt.show()
# #     img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #Convert image back to RGB from BGR
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# #     print(gray.shape)


#     data[index]=gray#img

In [ ]:
# train_csv['path'][train_csv['num_boxes']!=0][:data.shape[0]].iloc[1]

In [ ]:
# train_csv['annot_new'][train_csv['num_boxes']!=0][:data.shape[0]]

In [ ]:
# train_csv['annotations'][train_csv['num_boxes']!=0][:data.shape[0]]

In [ ]:
# data.shape

In [ ]:
# data[1].shape

In [ ]:
# np.swapaxes(data[1], 0, 2).shape


In [ ]:
# data[1]

In [ ]:
# img=cv2.imread(train_csv['path'][12679]) #Load image with most annotations #12679
# img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #Convert image back to RGB from BGR

# box_num=train_csv['num_boxes'][12679] #get num of boxes for that image

# for box in range(box_num): #Loop over num of boxes
#     annotations=train_csv['annot_new'].iloc[12679].split() #Get each number as an element of a list
#     cv2.rectangle(img,(int(annotations[4*box]),int(annotations[4*box+1])),
#                   (int(annotations[4*box])+int(annotations[4*box+2]),int(annotations[4*box+1])+int(annotations[4*box+3])), 
#                        (255,0,0), 2) #Add to the image one box at a time
#     cv2.putText(img,'Starfish',(int(annotations[4*box]),int(annotations[4*box+1])),cv2.FONT_HERSHEY_DUPLEX,0.6,(0,0,255),1)
#     #Put text above each rectangle
    
#Show image
# from PIL import Image
# img = Image.fromarray(data[1], 'RGB')
# # img.save('my.png')
# img.show()

# plt.figure(figsize= (10,10)) 
# plt.imshow(data[1],cmap='gray')
# plt.show()

In [ ]:
# data.shape

In [ ]:
# data_res=data.reshape(data.shape[0],data.shape[1]*data.shape[2])

In [ ]:
# labels=train_csv['annot_new'][train_csv['num_boxes']!=0][:data.shape[0]]
# from sklearn.tree import DecisionTreeClassifier
# clf_tree = DecisionTreeClassifier(criterion="entropy", max_depth=3, random_state=17)
# clf_tree.fit(data_res, labels)

In [ ]:
# data_pred=np.zeros((int(len(train_csv['path'][train_csv['num_boxes']!=0])/50),720,1280),dtype=np.uint8)

In [ ]:
# for index,path in enumerate(train_csv['path'][train_csv['num_boxes']!=0][data.shape[0]:2*data.shape[0]]):
# #     print(path)
#     img=cv2.imread(path) #Load image
# #     print(img.shape)
# #     plt.imshow(img)
# #     plt.show()
# #     img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #Convert image back to RGB from BGR
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# #     print(gray.shape)
#     data_pred[index]=gray#img

In [ ]:
# data_pred_fin=data_pred.reshape(data_pred.shape[0],data_pred.shape[1]*data_pred.shape[2])

In [ ]:
# data_pred.shape

In [ ]:
# preds=clf_tree.predict(data_pred_fin)
# preds.shape

In [ ]:
# preds[0]

In [ ]:
# preds

In [ ]:
# np.unique(preds)

In [ ]:
# labels_pred=train_csv['annot_new'][train_csv['num_boxes']!=0][data.shape[0]:2*data.shape[0]]
# labels_pred.shape

In [ ]:
# np.unique(labels_pred)

In [ ]:
# data_pred[0].shape

In [ ]:
# data.shape[0]

In [ ]:
# train_csv['path'][train_csv['num_boxes']!=0][data.shape[0]:2*data.shape[0]]

In [ ]:
# img=cv2.imread(data_pred[0]) #Load image with most annotations #12679
# cv2.imshow("image", data_pred[0])
# cv2.waitKey()
# img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #Convert image back to RGB from BGR

# box_num=train_csv['num_boxes'][12679] #get num of boxes for that image

# for box in range(box_num): #Loop over num of boxes
#     annotations=train_csv['annot_new'].iloc[12679].split() #Get each number as an element of a list
#     cv2.rectangle(img,(int(annotations[4*box]),int(annotations[4*box+1])),
#                   (int(annotations[4*box])+int(annotations[4*box+2]),int(annotations[4*box+1])+int(annotations[4*box+3])), 
#                        (255,0,0), 2) #Add to the image one box at a time
#     cv2.putText(img,'Starfish',(int(annotations[4*box]),int(annotations[4*box+1])),cv2.FONT_HERSHEY_DUPLEX,0.6,(0,0,255),1)
#     #Put text above each rectangle
    
#Show image
# from PIL import Image
# img = Image.fromarray(data[1], 'RGB')
# # img.save('my.png')
# img.show()

# plt.figure(figsize= (10,10)) 
# plt.imshow(img,cmap='gray')
# plt.show()

In [ ]:
# preds[0]

In [ ]:
# labels_pred.iloc[0]

## kNN - Not Working - A lot of features due to the "curse of dimensionality"

In [ ]:
# data_res.shape

In [ ]:
# labels.shape

In [ ]:
# data_pred_fin.shape

In [ ]:
# from sklearn.model_selection import StratifiedKFold, train_test_split
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.preprocessing import StandardScaler

In [ ]:
# knn = KNeighborsClassifier(n_neighbors=5)
#data_res, labels

In [ ]:
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(data_res)
# # X_holdout_scaled = scaler.transform(X_holdout)
# knn.fit(X_train_scaled, labels)

In [ ]:
# knn_pred = knn.predict(data_pred_fin)

In [ ]:
# knn_pred.shape

In [ ]:
# knn_pred

In [ ]:
# end2=time.time()

In [ ]:
# print('total time is {}'.format(end2-start))